<center><img src="../Picture Data/logo.png" alt="Header" style="width: 800px;"/></center>

@Copyright (C): 2010-2019, Shenzhen Yahboom Tech  
@Author: Malloy.Yuan  
@Date: 2019-07-17 10:10:02  
@LastEditors: Malloy.Yuan  
@LastEditTime: 2019-09-17 17:54:19  

## Import related packages to create camera instances

In [158]:
import cv2
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
import numpy as np



In [2]:
from jetbot import Camera, Robot
from jetbot import bgr8_to_jpeg
import PID
camera = Camera.instance(width=720, height=720)

In [4]:
SERVO_MID = 2180

## Create related control variables

In [6]:
global color_x, color_y, color_radius
color_x = color_y = color_radius = 0
global target_valuex
target_valuex = 2100
global target_valuey
target_valuey = 2048

## Create an array that stores HSV color gamut color classification data

In [7]:
import numpy as np
global color_lower
color_lower=np.array([156,43,46])
global color_upperv
color_upper = np.array([180, 255, 255])

## Creating a PID Control Instance

In [8]:
xservo_pid = PID.PositionalPID(1.9, 0.3, 0.35)
yservo_pid = PID.PositionalPID(1.5, 0.2, 0.3)

In [9]:
xservo_pid.SetStepSignal(150)

## Create a PTZ servo engine instance

In [10]:
from servoserial import ServoSerial
servo_device = ServoSerial() 

serial Open!


### Color recognition default array data

In [186]:
color_lower=np.array([156,43,46])
color_upper = np.array([180, 255, 255])
target_valuex = target_valuey = 2048
servo_device.Servo_serial_double_control(1, SERVO_MID, 2, 2048)

151
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08\x84\x00\n\x02\x08\x00\x00\n\x97'


### Set to identify red array data

In [176]:
color_lower=np.array([0,43,46])
color_upper = np.array([10, 255, 255])
target_valuex = target_valuey = 2048
servo_device.Servo_serial_double_control(1, 2048, 2, 2048)

27
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08\x00\x00\n\x02\x08\x00\x00\n\x1b'


### Set to identify yellow array data

In [13]:
color_lower=np.array([26,43,46])
color_upper = np.array([34, 255, 255])
target_valuex = 2100
target_valuey = 2048
servo_device.Servo_serial_double_control(1, 2048, 2, 2048)

27
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08\x00\x00\n\x02\x08\x00\x00\n\x1b'


### Set to identify blue array data

In [14]:
color_lower=np.array([100,43,46])
color_upper = np.array([124, 255, 255])
target_valuex = 2100
target_valuey = 2048
servo_device.Servo_serial_double_control(1, 2048, 2, 2048)

27
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08\x00\x00\n\x02\x08\x00\x00\n\x1b'


### Set to identify green array data

In [ ]:
color_lower=np.array([35,43,46])
color_upper = np.array([77, 255, 255])
target_valuex = 2100
target_valuey = 2048
servo_device.Servo_serial_double_control(1, 2048, 2, 2048)

### Set to identify orange array data

In [189]:
color_lower=np.array([11,43,46])
color_upper = np.array([25, 255, 255])
target_valuex = 2100
target_valuey = 2048
servo_device.Servo_serial_double_control(1, SERVO_MID, 2, 2048)

151
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08\x84\x00\n\x02\x08\x00\x00\n\x97'


In [164]:
def kmeans(X, k, max_iters=100):
    # 데이터 포인트의 수와 특성의 수 확인
    m, n = X.shape

    # 초기 중심 선택
    centroids = X[np.random.choice(m, k, replace=False)]

    for _ in range(max_iters):
       
        distances = np.linalg.norm(X[:, np.newaxis] - centroids, axis=2)
        labels = np.argmin(distances, axis=1)

        # 중심 재계산
        new_centroids = np.array([X[labels == j].mean(axis=0) for j in range(k)])

        # 수렴 여부 확인
        if np.all(centroids == new_centroids):
            break

        centroids = new_centroids

    return centroids, labels

## Main process of PTZ movement

In [17]:
robot = Robot()

## Create a display control

In [18]:
color_image = widgets.Image(format='jpeg', width=300, height=300)
display(color_image)

Image(value=b'', format='jpeg', height='300', width='300')

In [165]:
frame = camera.value
frame_c = frame.reshape(-1,3)
data = frame_c
k = 5
centroids, labels = kmeans(data, k)
labels, counts = np.unique(labels, return_counts=True)
cen = labels[counts.argmax()]
mask_color = centroids[cen]

In [179]:

frame = cv2.resize(frame, (300, 300))
frame_=cv2.GaussianBlur(frame,(5,5),0)                 
hsv=cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
mask=cv2.inRange(hsv,color_lower,color_upper)
mask=cv2.erode(mask,None,iterations=2)
mask=cv2.dilate(mask,None,iterations=2)
mask=cv2.GaussianBlur(mask,(3,3),0)
cnts=cv2.findContours(mask.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)[1]
if len(cnts)>0:
    cnt = max (cnts,key=cv2.contourArea)
    (color_x,color_y),color_radius=cv2.minEnclosingCircle(cnt)
if color_radius > 10:
    # Mark the detected color
    cv2.circle(frame,(int(color_x),int(color_y)),int(color_radius),(255,0,255),2)
color_image.value = bgr8_to_jpeg(frame)



error: OpenCV(4.1.1) /home/nvidia/host/build_opencv/nv_opencv/modules/imgproc/src/shapedescr.cpp:274: error: (-215:Assertion failed) npoints >= 0 && (depth == CV_32F || depth == CV_32S) in function 'contourArea'


In [194]:
while 1:
    frame = camera.value
    frame = cv2.resize(frame, (300, 300))
    frame_=cv2.GaussianBlur(frame,(5,5),0)                    
    hsv=cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    mask=cv2.inRange(hsv,color_lower,color_upper)  
    mask=cv2.erode(mask,None,iterations=2)
    mask=cv2.dilate(mask,None,iterations=2)
    mask=cv2.GaussianBlur(mask,(3,3),0)     
    cnts=cv2.findContours(mask.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)[-2] 
    if len(cnts)>0:
        cnt = max (cnts,key=cv2.contourArea)
        (color_x,color_y),color_radius=cv2.minEnclosingCircle(cnt)
    if color_radius > 10:
        # Mark the detected color
        cv2.circle(frame,(int(color_x),int(color_y)),int(color_radius),(255,0,255),2)  
        #Proportion-Integration-Differentiation
        xservo_pid.SystemOutput = color_x
        

        target_x = 150
        target_y = 150
        speed = 0.8
        turn = np.abs(target_x-color_x)/300
        if color_x < target_x - 60:
            robot.set_motors(speed-turn, speed)
        elif color_x > target_x + 60:
            robot.set_motors(speed, speed-turn)
        elif color_radius>50:
            robot.stop()
        else:
            robot.set_motors(speed, speed)
# Real-time return of image data for display
    color_image.value = bgr8_to_jpeg(frame)
    

KeyboardInterrupt: 

In [ ]:
xservo_pid.SetStepSignal(150)
        xservo_pid.SetInertiaTime(0.01, 0.006)
        target_valuex = int(2048+xservo_pid.SystemOutput)
        # Input Y axis direction parameter PID control input
        yservo_pid.SystemOutput = color_y
        yservo_pid.SetStepSignal(150)
        yservo_pid.SetInertiaTime(0.01, 0.006)
        target_valuey = int(2048+yservo_pid.SystemOutput)
        # Rotate the gimbal to the PID adjustment position
        servo_device.Servo_serial_double_control( 1, target_valuex, 2, target_valuey)

In [ ]:
camera.stop()

In [195]:
robot.stop()